# Caracterização de eletrofáceis por meio de inteligência artificial com abordagem supervisionada
## Verificação dos dados da fusão

- **Nome do Arquivo**: verificacao_fusao.ipynb
- **Autor**: Rafael Takeguma Goto
- **Versão**: 1.0
- **Data de Criação**: 01/07/2024
- **Data de Modificação**: 01/07/2024

### Descrição: 
Este notebook contém uma etapa adicional do trabalho referente à verificação dos dados fusão, a fim de certificar que o encaixe dos dados da fusão está correta.